### Analysis of Azure Pricing details 

This notebook will analyze service name = 'Virtual Machines'
source JSON file https://prices.azure.com/api/retail/prices?api-version=2021-10-01-preview&meterRegion=%E2%80%99primary%E2%80%99

In [61]:
# import json as json
import pandas as pd

# from collections import defaultdict
import requests

#### Access the API end point for 'Vitual Machine' and store JSON file at location ../data/raw/

In [40]:
r = requests.get(
    "https://prices.azure.com/api/retail/prices?$filter=serviceName eq 'Virtual Machines'"
)
with open("../../data/raw/azure_vm.json", "w") as fd:
    fd.write(r.text)

In [41]:
df = pd.read_json("../../data/raw/azure_vm.json")
df.head()

,BillingCurrency,CustomerEntityId,CustomerEntityType,Items,NextPageLink,Count
0,USD,Default,Retail,"{'currencyCode': 'USD', 'tierMinimumUnits': 0....",https://prices.azure.com:443/api/retail/prices...,100
1,USD,Default,Retail,"{'currencyCode': 'USD', 'tierMinimumUnits': 0....",https://prices.azure.com:443/api/retail/prices...,100
2,USD,Default,Retail,"{'currencyCode': 'USD', 'tierMinimumUnits': 0....",https://prices.azure.com:443/api/retail/prices...,100
3,USD,Default,Retail,"{'currencyCode': 'USD', 'tierMinimumUnits': 0....",https://prices.azure.com:443/api/retail/prices...,100
4,USD,Default,Retail,"{'currencyCode': 'USD', 'tierMinimumUnits': 0....",https://prices.azure.com:443/api/retail/prices...,100


#### Understand shape of data

In [42]:
print("Shape of the data\n\n", df.shape)
print("\n\n")
print("Is null?\n\n", df.isnull().sum())
print("\n\n")
print("Columnwise Datatype\n\n", df.dtypes)
df.describe()

Shape of the data

 (100, 6)



Is null?

 BillingCurrency       0
CustomerEntityId      0
CustomerEntityType    0
Items                 0
NextPageLink          0
Count                 0
dtype: int64



Columnwise Datatype

 BillingCurrency       object
CustomerEntityId      object
CustomerEntityType    object
Items                 object
NextPageLink          object
Count                  int64
dtype: object


,Count
count,100.0
mean,100.0
std,0.0
min,100.0
25%,100.0
50%,100.0
75%,100.0
max,100.0


#### Split data into multiple components, create series out of dataframes, List "Items" is carrying actual data items

In [43]:
df_items = df["Items"]
df_items = df_items.fillna("NA")

In [51]:
df_items

0     {'currencyCode': 'USD', 'tierMinimumUnits': 0....
1     {'currencyCode': 'USD', 'tierMinimumUnits': 0....
2     {'currencyCode': 'USD', 'tierMinimumUnits': 0....
3     {'currencyCode': 'USD', 'tierMinimumUnits': 0....
4     {'currencyCode': 'USD', 'tierMinimumUnits': 0....
                            ...                        
95    {'currencyCode': 'USD', 'tierMinimumUnits': 0....
96    {'currencyCode': 'USD', 'tierMinimumUnits': 0....
97    {'currencyCode': 'USD', 'tierMinimumUnits': 0....
98    {'currencyCode': 'USD', 'tierMinimumUnits': 0....
99    {'currencyCode': 'USD', 'tierMinimumUnits': 0....
Name: Items, Length: 100, dtype: object

#### Some times JSON file is missing reservationTerm or sometimes carry the values are 1 Year, 3 Years or onDemand
Below piece of code will determine if reservationTime key exists if not insert key with default value as onDemand

In [44]:
df_items_2 = pd.DataFrame()

for data in df_items:
    df_dictionary = pd.DataFrame([data])
    if "reservationTerm" in data:
        df_items_2 = pd.concat([df_items_2, df_dictionary], ignore_index=True)
    else:
        data["reservationTerm"] = "OnDemand"
        # df_dictionary = pd.DataFrame([data])
        df_items_2 = pd.concat([df_items_2, df_dictionary], ignore_index=True)

In [52]:
df_items_2 = df_items_2.fillna("NA")
df_items_2

,currencyCode,tierMinimumUnits,retailPrice,unitPrice,armRegionName,location,effectiveStartDate,effectiveEndDate,meterId,meterName,...,productName,skuName,serviceName,serviceId,serviceFamily,unitOfMeasure,type,isPrimaryMeterRegion,armSkuName,reservationTerm
0,USD,0.0,0.262752,0.262752,southindia,IN South,2022-03-01T00:00:00Z,2022-03-31T23:59:00Z,000009d0-057f-5f2b-b7e9-9e26add324a8,D14/DS14 Spot,...,Virtual Machines D Series Windows,D14 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_D14,NA
1,USD,0.0,0.542506,0.542506,southindia,IN South,2022-03-01T00:00:00Z,2022-03-31T23:59:00Z,000009d0-057f-5f2b-b7e9-9e26add324a8,D14/DS14 Spot,...,Virtual Machines D Series Windows,D14 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D14,NA
2,USD,0.0,0.865075,0.865075,eastasia,AP East,2022-03-01T00:00:00Z,2022-03-31T23:59:00Z,00014e7d-fff9-54dd-962e-4e992887ad3c,D64s v5 Spot,...,Virtual Machines Dsv5 Series Windows,Standard_D64s_v5 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_D64s_v5,NA
3,USD,0.0,1.468006,1.468006,eastasia,AP East,2022-03-01T00:00:00Z,2022-03-31T23:59:00Z,00014e7d-fff9-54dd-962e-4e992887ad3c,D64s v5 Spot,...,Virtual Machines Dsv5 Series Windows,Standard_D64s_v5 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D64s_v5,NA
4,USD,0.0,0.190000,0.190000,uknorth,UK North,2019-10-16T00:00:00Z,NA,0001e46a-9285-5fa8-b48a-240e307a24f7,A3 Spot,...,Virtual Machines A Series Windows,A3 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_A3,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,USD,0.0,0.180000,0.180000,uaecentral,AE Central,2021-11-01T00:00:00Z,NA,001fa4dd-bbe2-5502-b172-8a1af70ea8b8,D2ds v5,...,Virtual Machines Ddsv5 Series Windows,Standard_D2ds_v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_D2ds_v5,NA
96,USD,0.0,0.272000,0.272000,uaecentral,AE Central,2021-11-01T00:00:00Z,NA,001fa4dd-bbe2-5502-b172-8a1af70ea8b8,D2ds v5,...,Virtual Machines Ddsv5 Series Windows,Standard_D2ds_v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D2ds_v5,NA
97,USD,0.0,1.949000,1.949000,uksouth,UK South,2021-11-01T00:00:00Z,NA,001fa6ba-405f-5854-9dd5-5ecd8a7d7ef3,D48s v5 Low Priority,...,Virtual Machines Dsv5 Series Windows,Standard_D48s_v5 Low Priority,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D48s_v5,NA
98,USD,0.0,0.533000,0.533000,uksouth,UK South,2021-11-01T00:00:00Z,NA,001fa6ba-405f-5854-9dd5-5ecd8a7d7ef3,D48s v5 Low Priority,...,Virtual Machines Dsv5 Series Windows,Standard_D48s_v5 Low Priority,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,DevTestConsumption,True,Standard_D48s_v5,NA


#### Understand unique type, Typical values are Consumption and Reservation

In [53]:
print(pd.unique(df_items_2["type"]))

['DevTestConsumption' 'Consumption' 'Reservation']


In [54]:
print(pd.unique(df_items_2["reservationTerm"]))

['NA' '1 Year' '3 Years']


In [55]:
df_consumption = df_items_2.loc[(df_items_2["type"] == "Consumption")]
print(df_consumption)

   currencyCode  tierMinimumUnits  retailPrice  unitPrice       armRegionName  \
1           USD               0.0     0.542506   0.542506          southindia   
3           USD               0.0     1.468006   1.468006            eastasia   
4           USD               0.0     0.190000   0.190000             uknorth   
6           USD               0.0     0.075600   0.075600              westus   
7           USD               0.0     0.023600   0.023600           westindia   
8           USD               0.0     0.036547   0.036547         northeurope   
9           USD               0.0     1.504000   1.504000             eastus2   
12          USD               0.0     0.537088   0.537088  australiasoutheast   
13          USD               0.0     1.482000   1.482000      northcentralus   
14          USD               0.0     1.482000   1.482000      northcentralus   
15          USD               0.0     0.112000   0.112000         brazilsouth   
16          USD             

In [56]:
dfi_consumption = df_consumption[
    [
        "armRegionName",
        "productName",
        "skuName",
        "retailPrice",
        "currencyCode",
        "serviceName",
        "serviceFamily",
        "reservationTerm",
        "type",
    ]
]
dfi_consumption

,armRegionName,productName,skuName,retailPrice,currencyCode,serviceName,serviceFamily,reservationTerm,type
1,southindia,Virtual Machines D Series Windows,D14 Spot,0.542506,USD,Virtual Machines,Compute,NA,Consumption
3,eastasia,Virtual Machines Dsv5 Series Windows,Standard_D64s_v5 Spot,1.468006,USD,Virtual Machines,Compute,NA,Consumption
4,uknorth,Virtual Machines A Series Windows,A3 Spot,0.190000,USD,Virtual Machines,Compute,NA,Consumption
6,westus,Virtual Machines Ebdsv5 Series,Standard_E2bds_v5 Spot,0.075600,USD,Virtual Machines,Compute,NA,Consumption
7,westindia,Virtual Machines Dv4 Series,D2 v4 Low Priority,0.023600,USD,Virtual Machines,Compute,NA,Consumption
8,northeurope,Virtual Machines Esv5 Series,E2s v5 Spot,0.036547,USD,Virtual Machines,Compute,NA,Consumption
9,eastus2,Virtual Machines Dsv5 Series Windows,Standard_D16s_v5,1.504000,USD,Virtual Machines,Compute,NA,Consumption
12,australiasoutheast,Virtual Machines Eadsv5 Series Windows,E16-8ads v5 Spot,0.537088,USD,Virtual Machines,Compute,NA,Consumption
13,northcentralus,Virtual Machines Dv2 promo Series,D14 v2,1.482000,USD,Virtual Machines,Compute,NA,Consumption
14,northcentralus,Virtual Machines DSv2 promo Series,DS14 v2,1.482000,USD,Virtual Machines,Compute,NA,Consumption


#### Group by columns ('armRegionName','productName','skuName','serviceName','serviceFamily') to get region wise Max Price and Min Price

In [57]:
result_consumption = dfi_consumption.groupby(
    [
        "armRegionName",
        "productName",
        "skuName",
        "serviceName",
        "serviceFamily",
        "reservationTerm",
        "type",
    ]
).agg(maxPrice=("retailPrice", "max"), minPrice=("retailPrice", "min"))
result_consumption

maxPrice  \
armRegionName      productName                            skuName                        serviceName      serviceFamily reservationTerm type                     
australiacentral   Virtual Machines Edsv4 Series Windows  E8-2ds v4 Spot                 Virtual Machines Compute       NA              Consumption   0.184710   
australiacentral2  Virtual Machines Esv5 Series           E48s v5 Spot                   Virtual Machines Compute       NA              Consumption   1.449600   
australiaeast      Virtual Machines Esv4 Series Windows   E64-16s v4 Spot                Virtual Machines Compute       NA              Consumption   2.057530   
australiasoutheast Virtual Machines Eadsv5 Series Windows E16-8ads v5 Spot               Virtual Machines Compute       NA              Consumption   0.537088   
                   Virtual Machines F Series Windows      F16 Spot                       Virtual Machines Compute       NA              Consumption   0.403024   
brazilsouth        Virtual Machines A Series Basic        A2                             Virtual Machines Compute       NA              Consumption   0.112000   
                   Virtual Machines MSv2 Series Windows   M416s v2 Spot                  Virtual Machines Compute       NA              Consumption  17.373000   
canadacentral      Virtual Machines D Series Windows      D13 Low Priority               Virtual Machines Compute       NA              Consumption   0.456000   
                   Virtual Machines DS Series Windows     DS13 Low Priority              Virtual Machines Compute       NA              Consumption   0.456000   
                   Virtual Machines Edsv4 Series Windows  E20ds v4 Low Priority          Virtual Machines Compute       NA              Consumption   1.000000   
canadaeast         Virtual Machines Esv5 Series           E4-2s v5 Low Priority          Virtual Machines Compute       NA              Consumption   0.055200   
                                                          E4s v5 Low Priority            Virtual Machines Compute       NA              Consumption   0.055200   
centralus          Virtual Machines A Series Windows      A3 Spot                        Virtual Machines Compute       NA              Consumption   0.054000   
                   Virtual Machines Esv4 Series           E8-2s v4 Low Priority          Virtual Machines Compute       NA              Consumption   0.114000   
eastasia           Virtual Machines Dsv5 Series Windows   Standard_D64s_v5 Spot          Virtual Machines Compute       NA              Consumption   1.468006   
eastus             DCasv5-series Linux                    Standard_DC8as_v5 Low Priority Virtual Machines Compute       NA              Consumption   0.108000   
eastus2            Virtual Machines Dsv5 Series Windows   Standard_D16s_v5               Virtual Machines Compute       NA              Consumption   1.504000   
                   Virtual Machines Eadsv5 Series         E8-4ads v5 Low Priority        Virtual Machines Compute       NA              Consumption   0.105000   
                   Virtual Machines MdSv2 Series Windows  M192idms_v2                    Virtual Machines Compute       NA              Consumption  40.896000   
japaneast          Virtual Machines A Series Windows      A3                             Virtual Machines Compute       NA              Consumption   0.424000   
                   Virtual Machines Edsv5 Series          E104ids v5 Low Priority        Virtual Machines Compute       NA              Consumption   1.991000   
                   Virtual Machines Edv5 Series           Standard_E20d_v5 Low Priority  Virtual Machines Compute       NA              Consumption   0.348000   
japanwest          Virtual Machines Ddsv5 Series Windows  Standard_D16ds_v5 Spot         Virtual Machines Compute       NA              Consumption   0.389939   
                   Virtual Machines Esv4 Series Windows   E64s v4                        Virtual Mac

#### Data analysis to understand service prices for type Reservation

In [58]:
df_reservation = df_items_2.loc[(df_items_2["type"] == "Reservation")]
df_reservation

,currencyCode,tierMinimumUnits,retailPrice,unitPrice,armRegionName,location,effectiveStartDate,effectiveEndDate,meterId,meterName,...,productName,skuName,serviceName,serviceId,serviceFamily,unitOfMeasure,type,isPrimaryMeterRegion,armSkuName,reservationTerm
22,USD,0.0,18689.0,18689.0,westus2,US West 2,2021-11-01T00:00:00Z,NA,000aeefc-dae5-5264-8f64-d0838e160478,D64ds v5,...,Virtual Machines Ddsv5 Series,Standard_D64ds_v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_D64ds_v5,1 Year
24,USD,0.0,36111.0,36111.0,westus2,US West 2,2021-11-01T00:00:00Z,NA,000aeefc-dae5-5264-8f64-d0838e160478,D64ds v5,...,Virtual Machines Ddsv5 Series,Standard_D64ds_v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_D64ds_v5,3 Years
28,USD,0.0,77890.0,77890.0,koreasouth,KR South,2021-12-01T00:00:00Z,NA,000c0948-0687-52ed-8efa-32a3cfa6c9ed,Easv4 Type2,...,Easv4 Series Dedicated Host,Easv4 Type2,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Easv4_Type2,3 Years
29,USD,0.0,39231.0,39231.0,koreasouth,KR South,2021-12-01T00:00:00Z,NA,000c0948-0687-52ed-8efa-32a3cfa6c9ed,Easv4 Type2,...,Easv4 Series Dedicated Host,Easv4 Type2,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Easv4_Type2,1 Year
46,USD,0.0,20012.0,20012.0,usgovarizona,US Gov AZ,2020-08-01T00:00:00Z,NA,0014a546-6717-582f-9680-e510e91eae91,D64s v4,...,Virtual Machines Dsv4 Series,D64s v4,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_D64s_v4,1 Year
48,USD,0.0,38667.0,38667.0,usgovarizona,US Gov AZ,2020-08-01T00:00:00Z,NA,0014a546-6717-582f-9680-e510e91eae91,D64s v4,...,Virtual Machines Dsv4 Series,D64s v4,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_D64s_v4,3 Years
56,USD,0.0,48318.0,48318.0,southcentralus,US South Central,2020-08-01T00:00:00Z,NA,0016083a-928f-56fd-8eeb-39287dcf676d,E64 v4,...,Virtual Machines Ev4 Series,E64 v4,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_E64_v4,3 Years
57,USD,0.0,25007.0,25007.0,southcentralus,US South Central,2020-08-01T00:00:00Z,NA,0016083a-928f-56fd-8eeb-39287dcf676d,E64 v4,...,Virtual Machines Ev4 Series,E64 v4,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_E64_v4,1 Year
75,USD,0.0,34980.0,34980.0,northeurope,EU North,2021-11-01T00:00:00Z,NA,001a6f90-bc8f-572e-b7bc-f95af3413781,E96 v5,...,Virtual Machines Ev5 Series,Standard_E96_v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_E96_v5,1 Year
76,USD,0.0,67588.0,67588.0,northeurope,EU North,2021-11-01T00:00:00Z,NA,001a6f90-bc8f-572e-b7bc-f95af3413781,E96 v5,...,Virtual Machines Ev5 Series,Standard_E96_v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_E96_v5,3 Years


In [59]:
dfi_reservation = df_reservation[
    [
        "armRegionName",
        "productName",
        "skuName",
        "retailPrice",
        "currencyCode",
        "serviceName",
        "serviceFamily",
        "reservationTerm",
        "type",
    ]
]
dfi_reservation

,armRegionName,productName,skuName,retailPrice,currencyCode,serviceName,serviceFamily,reservationTerm,type
22,westus2,Virtual Machines Ddsv5 Series,Standard_D64ds_v5,18689.0,USD,Virtual Machines,Compute,1 Year,Reservation
24,westus2,Virtual Machines Ddsv5 Series,Standard_D64ds_v5,36111.0,USD,Virtual Machines,Compute,3 Years,Reservation
28,koreasouth,Easv4 Series Dedicated Host,Easv4 Type2,77890.0,USD,Virtual Machines,Compute,3 Years,Reservation
29,koreasouth,Easv4 Series Dedicated Host,Easv4 Type2,39231.0,USD,Virtual Machines,Compute,1 Year,Reservation
46,usgovarizona,Virtual Machines Dsv4 Series,D64s v4,20012.0,USD,Virtual Machines,Compute,1 Year,Reservation
48,usgovarizona,Virtual Machines Dsv4 Series,D64s v4,38667.0,USD,Virtual Machines,Compute,3 Years,Reservation
56,southcentralus,Virtual Machines Ev4 Series,E64 v4,48318.0,USD,Virtual Machines,Compute,3 Years,Reservation
57,southcentralus,Virtual Machines Ev4 Series,E64 v4,25007.0,USD,Virtual Machines,Compute,1 Year,Reservation
75,northeurope,Virtual Machines Ev5 Series,Standard_E96_v5,34980.0,USD,Virtual Machines,Compute,1 Year,Reservation
76,northeurope,Virtual Machines Ev5 Series,Standard_E96_v5,67588.0,USD,Virtual Machines,Compute,3 Years,Reservation


In [60]:
result_reservation = dfi_reservation.groupby(
    [
        "armRegionName",
        "productName",
        "skuName",
        "serviceName",
        "serviceFamily",
        "reservationTerm",
        "type",
    ]
).agg(maxPrice=("retailPrice", "max"), minPrice=("retailPrice", "min"))
result_reservation

maxPrice  \
armRegionName  productName                   skuName           serviceName      serviceFamily reservationTerm type                    
koreasouth     Easv4 Series Dedicated Host   Easv4 Type2       Virtual Machines Compute       1 Year          Reservation   39231.0   
                                                                                              3 Years         Reservation   77890.0   
northeurope    Virtual Machines Ev5 Series   Standard_E96_v5   Virtual Machines Compute       1 Year          Reservation   34980.0   
                                                                                              3 Years         Reservation   67588.0   
southcentralus Virtual Machines Ev4 Series   E64 v4            Virtual Machines Compute       1 Year          Reservation   25007.0   
                                                                                              3 Years         Reservation   48318.0   
usgovarizona   Virtual Machines Dsv4 Series  D64s v4           Virtual Machines Compute       1 Year          Reservation   20012.0   
                                                                                              3 Years         Reservation   38667.0   
westus2        Virtual Machines Ddsv5 Series Standard_D64ds_v5 Virtual Machines Compute       1 Year          Reservation   18689.0   
                                                                                              3 Years         Reservation   36111.0   

                                                                                                                           minPrice  
armRegionName  productName                   skuName           serviceName      serviceFamily reservationTerm type                   
koreasouth     Easv4 Series Dedicated Host   Easv4 Type2       Virtual Machines Compute       1 Year          Reservation   39231.0  
                                                                                              3 Years         Reservation   77890.0  
northeurope    Virtual Machines Ev5 Series   Standard_E96_v5   Virtual Machines Compute       1 Year          Reservation   34980.0  
                                                                                              3 Years         Reservation   67588.0  
southcentralus Virtual Machines Ev4 Series   E64 v4            Virtual Machines Compute       1 Year          Reservation   25007.0  
                                                                                              3 Years         Reservation   48318.0  
usgovarizona   Virtual Machines Dsv4 Series  D64s v4           Virtual Machines Compute       1 Year          Reservation   20012.0  
                                                                                              3 Years         Reservation   38667.0  
westus2        Virtual Machines Ddsv5 Series Standard_D64ds_v5 Virtual Machines Compute       1 Year          Reservation   18689.0  
                                                                                              3 Years         Reservation   36111.0

#### Conclusion - 
For the type Consumption In most of the cases across regions, there is slight variation between min and max price

For the type Reservation prices are fixed. Max min price analysis is not required.